In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%cd 'drive/My Drive'


/content/drive/My Drive


#needed libraries

In [3]:
pip install -U turicreate

Requirement already up-to-date: turicreate in /usr/local/lib/python3.6/dist-packages (6.2)


In [0]:
import turicreate as turi
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


#load data

In [0]:
#define dataset locations
ids_file='https://static.turi.com/datasets/millionsong/10000.txt' # is a file loc with song ids, user ids and listen count
metadata_loc='https://static.turi.com/datasets/millionsong/song_data.csv' # is a file loc which contains song names, song ids, artist details

#combine both of these dataframes such that there exists 1 dataframe with prominence given to song id, user id and listen count
song_pd=pd.read_table(ids_file,header=None)
song_pd.columns=['user_id','song_id','listen_count']
metadata_pd=pd.read_csv(metadata_loc)

song_data=pd.merge(song_pd,metadata_pd.drop_duplicates(['song_id']), on='song_id', how='left')

#Data visualization

In [0]:
#define a new column entry called 'song' that combines title and artist name
song_data['song']=song_data['title'].map(str)+"-"+song_data['artist_name'].map(str)

In [12]:
users = song_data['user_id'].unique()
print(len(users))

76353


# Get the ten popular songs for Popularity Recommender

In [19]:
#get the aggregate listen count and percentage value for each song and sort the data
songs_count=song_data.groupby(['song']).agg({'listen_count':'count'}).reset_index()
grouped_sum=songs_count['listen_count'].sum()
songs_count['percentage']=songs_count['listen_count']/grouped_sum
songs_count.sort_values(['listen_count','song'], ascending=[0,1])
songs_count
popular = songs_count.sort_values(by = 'listen_count')
popular
popular.tail(10)

,song,listen_count,percentage
8847,Tive Sim-Cartola,4548,0.002274
3485,Hey_ Soul Sister-Train,4758,0.002379
2717,Fireflies-Charttraxx Karaoke,4795,0.002397
3613,Horn Concerto No. 4 in E flat K495: II. Romanc...,5385,0.002693
7115,Secrets-OneRepublic,5841,0.002920
6774,Revelry-Kings Of Leon,6145,0.003073
9877,You're The One-Dwight Yoakam,6412,0.003206
2068,Dog Days Are Over (Radio Edit)-Florence + The ...,6949,0.003475
9084,Undo-Björk,7032,0.003516
7127,Sehr kosmisch-Harmonia,8277,0.004138


##convert pandas Data frame to SFrame

In [0]:
song_data=turi.SFrame(song_data)

#split dataset

In [0]:
train_data,test_data=song_data.random_split(0.8,seed=101)

#bulid personalized model


In [23]:
perosnalized_model = turi.item_similarity_recommender.create(train_data,user_id='user_id',item_id='song')


Warning: Ignoring columns song_id, listen_count, title, release, artist_name, year;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 1600134 observations with 76140 users and 9953 items.

...

| 3.80s                               | 100              | 9953            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.91211s

##personalied recommendation for user 0

In [24]:
perosnalized_model.recommend([users[0]])


user_id,song,score,rank
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Quiet Houses-Fleet Foxes,0.025714509189128876,1
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Meadowlarks-Fleet Foxes,0.024423322081565856,2
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Tiger Mountain PeasantSong-Fleet Foxes ...,0.023262405395507814,3
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Sun It Rises-Fleet Foxes,0.023195575177669524,4
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Heard Them Stirring-FleetFoxes ...,0.02283104956150055,5
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Oliver James-Fleet Foxes,0.022458715736865996,6
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Your Protector-FleetFoxes ...,0.021642793715000153,7
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Great Indoors-John Mayer,0.02155410647392273,8
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Drops In The River-FleetFoxes ...,0.01965464949607849,9
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,Belle-Jack Johnson,0.018890346586704253,10


## get similar song to song Quiet Houses-Fleet Foxes	

In [26]:
perosnalized_model.get_similar_items(['Quiet Houses-Fleet Foxes'])


song,similar,score,rank
Quiet Houses-Fleet Foxes,Meadowlarks-Fleet Foxes,0.41198503971099854,1
Quiet Houses-Fleet Foxes,Heard Them Stirring-FleetFoxes ...,0.3621399402618408,2
Quiet Houses-Fleet Foxes,Your Protector-FleetFoxes ...,0.3595890402793884,3
Quiet Houses-Fleet Foxes,He Doesn't Know Why-FleetFoxes ...,0.3546326160430908,4
Quiet Houses-Fleet Foxes,Oliver James-Fleet Foxes,0.3538960814476013,5
Quiet Houses-Fleet Foxes,Tiger Mountain PeasantSong-Fleet Foxes ...,0.34954404830932617,6
Quiet Houses-Fleet Foxes,Ragged Wood-Fleet Foxes,0.32692307233810425,7
Quiet Houses-Fleet Foxes,Sun It Rises-Fleet Foxes,0.3020833134651184,8
Quiet Houses-Fleet Foxes,White Winter Hymnal-FleetFoxes ...,0.21861469745635986,9
Quiet Houses-Fleet Foxes,Sun Giant-Fleet Foxes,0.20279717445373535,10
